In [1]:
!pip install indic-nlp-library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.5 MB/s eta 0:00:00


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoTokenizer
import torch
import torch.nn.functional as F

In [3]:
from indicnlp.tokenize import indic_tokenize

In [4]:
df=pd.read_csv("/kaggle/input/english-hindi-ml/Sentence pairs in English-Hindi - 2025-02-11.tsv",sep="\t",header=None,names=["SrcSentID","SrcSent","DestSentID","DestSent"])

In [5]:
df.head()

,SrcSentID,SrcSent,DestSentID,DestSent
0,1282,Muiriel is 20 now.,485968,म्यूरियल अब बीस साल की हो गई है।
1,1282,Muiriel is 20 now.,2060319,म्यूरियल अब बीस साल की है।
2,1294,Education in this world disappoints me.,485564,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,1302,That won't happen.,2060320,वैसा नहीं होगा।
4,1308,I miss you.,2060321,मुझें तुम्हारी याद आ रही है।


In [6]:
df.shape

(13182, 4)

In [7]:
df.drop(labels=[df.columns[0],df.columns[2]],axis=1,inplace=True)

In [8]:
df.head()

,SrcSent,DestSent
0,Muiriel is 20 now.,म्यूरियल अब बीस साल की हो गई है।
1,Muiriel is 20 now.,म्यूरियल अब बीस साल की है।
2,Education in this world disappoints me.,मैं इस दुनिया में शिक्षा पर बहुत निराश हूँ।
3,That won't happen.,वैसा नहीं होगा।
4,I miss you.,मुझें तुम्हारी याद आ रही है।


In [9]:
src_sent_tokenizer=AutoTokenizer.from_pretrained("google-T5/T5-base")

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [10]:
df["SrcSent"]=df["SrcSent"].apply(lambda x: src_sent_tokenizer.tokenize(x))

In [11]:
df["DestSent"]=df["DestSent"].apply(lambda x: indic_tokenize.trivial_tokenize(x,lang='hi'))

In [12]:
df.head()

,SrcSent,DestSent
0,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, हो, गई, है, ।]"
1,"[▁Mu, i, riel, ▁is, ▁20, ▁now, .]","[म्यूरियल, अब, बीस, साल, की, है, ।]"
2,"[▁Education, ▁in, ▁this, ▁world, ▁disappoint, ...","[मैं, इस, दुनिया, में, शिक्षा, पर, बहुत, निराश..."
3,"[▁That, ▁won, ', t, ▁happen, .]","[वैसा, नहीं, होगा, ।]"
4,"[▁I, ▁miss, ▁you, .]","[मुझें, तुम्हारी, याद, आ, रही, है, ।]"


In [13]:
df["SrcSent"]=df["SrcSent"].apply(src_sent_tokenizer.convert_tokens_to_ids)

In [14]:
Vs=src_sent_tokenizer.get_vocab()
Vs
len(Vs)

32100

In [15]:
Vs['stepped']

14417

In [16]:
hindi_vocab=set()
for tokenized_hindi_sent in df["DestSent"]:
    hindi_vocab.update(tokenized_hindi_sent)

In [17]:
hindi_vocab
len(hindi_vocab)

7069

In [18]:
Vd=dict()
for idx,token in enumerate(hindi_vocab):
    Vd[token]=idx+3

Vd["<PAD>"]=0
Vd["<SOS>"]=1
Vd["<EOS>"]=2

In [19]:
hindi_idx2vocab=dict(zip(Vd.values(),Vd.keys()))
print(hindi_idx2vocab)

{3: 'दिवाला', 4: 'सिखाए', 5: 'अक्साई', 6: 'डराने', 7: 'पंछी', 8: 'अठारहवाँ', 9: 'प्रोजेक्ट', 10: 'मुस्कुराए', 11: 'व्यापारी', 12: 'मारना', 13: 'चिह्नित', 14: 'मज़े', 15: 'जर्मनी', 16: 'अम्रीकी', 17: 'वापस', 18: 'जवाहरलाल', 19: 'बर्बाद', 20: 'करुँगी', 21: 'बैठूं', 22: 'ज़िंदाबाद', 23: 'दाँत', 24: 'बताता', 25: 'भरता', 26: 'दोबारा', 27: 'आर्नोल्ड', 28: 'सर्दियों', 29: 'पतला', 30: 'लॉटरी', 31: 'नीला', 32: 'सौन्\u200dदर्यवान', 33: 'भागा', 34: 'नेटफ़्लिक्स', 35: 'जाता', 36: 'बोलोगे', 37: 'तोता', 38: 'महँगा', 39: 'चुम्मा', 40: 'लंड', 41: 'पैंट', 42: 'गवा', 43: 'कंपनी', 44: 'सुविचरों', 45: 'कहोगे', 46: 'प्रथम', 47: 'मसाओ', 48: 'टेबल', 49: 'निकालने', 50: 'जन्म', 51: 'मुग़ल', 52: 'दादीजी', 53: 'कत्ल', 54: 'ख़ूनी', 55: 'कहने', 56: 'अपराधों', 57: 'मछलियाँ', 58: 'हिटलर', 59: 'तेरे', 60: 'प्रेत', 61: 'इंतजार', 62: 'स्वागतम्', 63: 'लड़ी', 64: 'ढूँढूं', 65: 'आईएगा', 66: 'कितनी', 67: 'रोकना', 68: 'बहने', 69: 'ज़ू', 70: 'ग्रामीण', 71: 'मोबाइल', 72: 'दूध', 73: 'तीर', 74: 'सिखाओगी', 75: 'रोगी', 76: 'घनिष्

In [20]:
Vd
len(Vd)

7072

In [21]:
def convert_hindi_tokens_to_ids(hindi_sent):
    return [Vd[token] for token in hindi_sent]

In [22]:
df["DestSent"]=df["DestSent"].apply(lambda x:convert_hindi_tokens_to_ids(x))

In [23]:
df.head()

,SrcSent,DestSent
0,"[4159, 23, 14018, 19, 460, 230, 5]","[983, 5626, 4056, 5157, 5652, 6409, 3972, 2256..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[983, 5626, 4056, 5157, 5652, 2256, 6726]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[5441, 951, 2804, 3353, 1588, 5321, 4486, 6514..."
3,"[466, 751, 31, 17, 1837, 5]","[805, 2042, 1668, 6726]"
4,"[27, 3041, 25, 5]","[4227, 1786, 1862, 5479, 2245, 2256, 6726]"


In [24]:
def insert_sos_token_id(hindi_sent_token):
    return [1]+hindi_sent_token

def insert_eos_token_id(hindi_sent_token):
    return hindi_sent_token+[2]

In [25]:
df["DestSentInput"]=df["DestSent"].apply(lambda x:insert_sos_token_id(x))

In [26]:
df["DestSentLabel"]=df["DestSent"].apply(lambda x:insert_eos_token_id(x))

In [27]:
df.head()

,SrcSent,DestSent,DestSentInput,DestSentLabel
0,"[4159, 23, 14018, 19, 460, 230, 5]","[983, 5626, 4056, 5157, 5652, 6409, 3972, 2256...","[1, 983, 5626, 4056, 5157, 5652, 6409, 3972, 2...","[983, 5626, 4056, 5157, 5652, 6409, 3972, 2256..."
1,"[4159, 23, 14018, 19, 460, 230, 5]","[983, 5626, 4056, 5157, 5652, 2256, 6726]","[1, 983, 5626, 4056, 5157, 5652, 2256, 6726]","[983, 5626, 4056, 5157, 5652, 2256, 6726, 2]"
2,"[2855, 16, 48, 296, 26963, 7, 140, 5]","[5441, 951, 2804, 3353, 1588, 5321, 4486, 6514...","[1, 5441, 951, 2804, 3353, 1588, 5321, 4486, 6...","[5441, 951, 2804, 3353, 1588, 5321, 4486, 6514..."
3,"[466, 751, 31, 17, 1837, 5]","[805, 2042, 1668, 6726]","[1, 805, 2042, 1668, 6726]","[805, 2042, 1668, 6726, 2]"
4,"[27, 3041, 25, 5]","[4227, 1786, 1862, 5479, 2245, 2256, 6726]","[1, 4227, 1786, 1862, 5479, 2245, 2256, 6726]","[4227, 1786, 1862, 5479, 2245, 2256, 6726, 2]"


In [28]:
df.drop(labels=[df.columns[1]],axis=1,inplace=True)

In [29]:
X=list(df["SrcSent"])
Y_input=list(df["DestSentInput"])
Y_label=list(df["DestSentLabel"])

In [30]:
X_tensor=[torch.tensor(tokenized_eng_sent_ids) for tokenized_eng_sent_ids in X]

In [31]:
Y_input_tensor=[torch.tensor(tokenized_hindi_sent_ids)for tokenized_hindi_sent_ids in Y_input]

In [32]:
Y_label_tensor=[torch.tensor(tokenized_hindi_sent_ids)for tokenized_hindi_sent_ids in Y_label]

In [33]:
X_padded=torch.nn.utils.rnn.pad_sequence(X_tensor,batch_first=True)

In [34]:
Y_padded_input=torch.nn.utils.rnn.pad_sequence(Y_input_tensor,batch_first=True)

In [35]:
Y_padded_label=torch.nn.utils.rnn.pad_sequence(Y_label_tensor,batch_first=True)

In [36]:
Ns=X_padded.shape[1]
Nd=Y_padded_label.shape[1]

In [37]:
Ns

68

In [38]:
Nd

68

In [39]:
class Encoder(torch.nn.Module):
    def __init__(self,src_lang_vocab_size,word_embedding_dim):
        super(Encoder,self).__init__()
        self.first_embedding_layer=torch.nn.Embedding(num_embeddings=src_lang_vocab_size, embedding_dim=word_embedding_dim)
        self.second_lstm_layer=torch.nn.LSTM(input_size=word_embedding_dim,hidden_size=word_embedding_dim,batch_first=True)


    def forward(self,X_padded_mini_batch):
        first_embedding_layer_out=self.first_embedding_layer(X_padded_mini_batch)
        encoder_output,(final_encoder_output,final_cell_state)=self.second_lstm_layer(first_embedding_layer_out)
        return encoder_output,(final_encoder_output,final_cell_state)

In [40]:
class Decoder(torch.nn.Module):
    def __init__(self,dest_lang_vocab_size,word_embedding_dim):
        super(Decoder,self).__init__()
        self.first_embedding_layer=torch.nn.Embedding(num_embeddings=dest_lang_vocab_size, embedding_dim=word_embedding_dim)
        self.second_lstm_layer=torch.nn.LSTM(input_size=word_embedding_dim,hidden_size=word_embedding_dim,batch_first=True)

        self.prediction_layer=torch.nn.Linear(in_features=word_embedding_dim,out_features=dest_lang_vocab_size)
        # self.prediction_layer_activation=torch.nn.Softmax(dim=2)

    def forward(self,Y_padded_input_mini_batch,final_encoder_output,final_cell_state):
        first_embedding_layer_out=self.first_embedding_layer(Y_padded_input_mini_batch)
        decoder_lstm_layer_out,(final_decoder_lstm_layer_out,final_cell_state)=self.second_lstm_layer(first_embedding_layer_out,
                                                                                                     (final_encoder_output,
                                                                                                     final_cell_state))
        prediction=self.prediction_layer(decoder_lstm_layer_out)
        return prediction, (final_decoder_lstm_layer_out,final_cell_state)

In [41]:
class Seq2SeqEncDec(torch.nn.Module):
    def __init__(self,src_lang_vocab_size,dest_lang_vocab_size,word_embedding_dim):
        super(Seq2SeqEncDec,self).__init__()

        self.encoder=Encoder(src_lang_vocab_size,word_embedding_dim)
        self.decoder=Decoder(dest_lang_vocab_size,word_embedding_dim)

    def forward(self,X_padded_mini_batch,Y_padded_input_mini_batch):
        encoder_output,(final_encoder_output,final_cell_state)=self.encoder(X_padded_mini_batch)
        y_hat_mini_batch=self.decoder(Y_padded_input_mini_batch,final_encoder_output,final_cell_state)

        return y_hat_mini_batch

In [42]:
if torch.cuda.is_available():
    device=torch.device("cuda")
else:
    device=torch.device("cpu")
print(device)

cuda


In [43]:
X_padded_train=X_padded[0:13000]
Y_padded_input_train=Y_padded_input[0:13000]
Y_padded_label_train=Y_padded_label[0:13000]

X_padded_test=X_padded[13000:]
Y_padded_input_test=Y_padded_input[13000:]
Y_padded_label_test=Y_padded_label[13000:]


In [44]:
network = Seq2SeqEncDec(len(Vs),len(Vd),128).to(device)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam(network.parameters())
num_epochs = 500
mb_size = 65

for epoch in range(num_epochs):
    for i in range(X_padded_train.shape[0]//mb_size):

        X_train_mb = X_padded_train[i*mb_size:(i+1)*mb_size]
        Y_input_mb = Y_padded_input_train[i*mb_size:(i+1)*mb_size]
        Y_label_mb = Y_padded_label_train[i*mb_size:(i+1)*mb_size]
        Y_label_mb = Y_label_mb.reshape(Y_label_mb.shape[0]*Y_label_mb.shape[1],)
        
        X_train_mb, Y_input_mb, Y_label_mb = X_train_mb.to(device), Y_input_mb.to(device), Y_label_mb.to(device)

        y_hat_train_mb = network(X_train_mb,Y_input_mb)
        y_hat_train_mb = y_hat_train_mb[0]
        y_hat_train_mb = y_hat_train_mb.reshape(y_hat_train_mb.shape[0]*y_hat_train_mb.shape[1],
                                                y_hat_train_mb.shape[2])

        loss_fn_value = loss_fn(y_hat_train_mb,Y_label_mb)

        loss_fn_value.backward()
        torch.nn.utils.clip_grad_norm_(network.parameters(),max_norm=1.0)
        optimizer.step()
        optimizer.zero_grad()

    print("i # {}, Time Step # {}, Loss Value = {}".format(epoch,i,loss_fn_value))

i # 0, Time Step # 199, Loss Value = 4.917413711547852
i # 1, Time Step # 199, Loss Value = 4.613677978515625
i # 2, Time Step # 199, Loss Value = 4.409133434295654
i # 3, Time Step # 199, Loss Value = 4.227604389190674
i # 4, Time Step # 199, Loss Value = 4.073513507843018
i # 5, Time Step # 199, Loss Value = 3.9372286796569824
i # 6, Time Step # 199, Loss Value = 3.817857503890991
i # 7, Time Step # 199, Loss Value = 3.709331512451172
i # 8, Time Step # 199, Loss Value = 3.59694242477417
i # 9, Time Step # 199, Loss Value = 3.494412422180176
i # 10, Time Step # 199, Loss Value = 3.3921124935150146
i # 11, Time Step # 199, Loss Value = 3.2939774990081787
i # 12, Time Step # 199, Loss Value = 3.19881010055542
i # 13, Time Step # 199, Loss Value = 3.1056652069091797
i # 14, Time Step # 199, Loss Value = 3.0185515880584717
i # 15, Time Step # 199, Loss Value = 2.9284629821777344
i # 16, Time Step # 199, Loss Value = 2.846647262573242
i # 17, Time Step # 199, Loss Value = 2.77014231681823

## Model is trained now

In [45]:
torch.save(network.state_dict(),"model.pth")

## **Inference Logic**

In [46]:
def generate_translation(eng_sentence):

    tokenized_eng_sentence = src_sent_tokenizer.tokenize(eng_sentence)
    token_ids = src_sent_tokenizer.convert_tokens_to_ids(tokenized_eng_sentence)
    token_ids_tensor = torch.tensor(token_ids)
    token_ids_tensor = torch.unsqueeze(token_ids_tensor,0)

    if torch.cuda.is_available():
        device = torch.device("cuda")
        token_ids_tensor = token_ids_tensor.to(device)

    encoder_outputs,(final_encoder_output,final_candidate_cell_state) = network.encoder(token_ids_tensor)
    decoder_first_time_step_input = torch.tensor([[1]])

    if torch.cuda.is_available():
        encoder_outputs = encoder_outputs.to(device)
        final_encoder_output = final_encoder_output.to(device)
        final_candidate_cell_state = final_candidate_cell_state.to(device)
        decoder_first_time_step_input = decoder_first_time_step_input.to(device)

    decoder_first_time_step_output, (hidden_decoder_output, hidden_decoder_cell_state) = network.decoder(decoder_first_time_step_input,
                                                                          final_encoder_output,
                                                                          final_candidate_cell_state)

    generated_token_id = torch.argmax(F.softmax(decoder_first_time_step_output[:,0,:],dim=1),1)
    generated_token_id = torch.unsqueeze(generated_token_id,1)

    hindi_translated_sentence = str()
    hindi_translated_sentence += " " + hindi_idx2vocab[generated_token_id.item()]

    if torch.cuda.is_available():
        generated_token_id = generated_token_id.to(device)
        hidden_decoder_output = hidden_decoder_output.to(device)
        hidden_decoder_cell_state = hidden_decoder_cell_state.to(device)
        
    for i in range(Nd-1):
        
        decoder_first_time_step_output, (hidden_decoder_output, hidden_decoder_cell_state) = network.decoder(generated_token_id,
                                                                                                hidden_decoder_output,
                                                                                                hidden_decoder_cell_state)
        generated_token_id = torch.argmax(F.softmax(decoder_first_time_step_output[:,0,:],dim=1),1)
        generated_token_id = torch.unsqueeze(generated_token_id,1)

        if torch.cuda.is_available():
            generated_token_id = generated_token_id.to(device)
            hidden_decoder_output = hidden_decoder_output.to(device)
            hidden_decoder_cell_state = hidden_decoder_cell_state.to(device)

        if generated_token_id.item() == Vd["<EOS>"]:
            break

        hindi_translated_sentence += " " + hindi_idx2vocab[generated_token_id.item()]

    return hindi_translated_sentence

In [1]:
generate_translation("hello")

NameError: name 'generate_translation' is not defined

## Deployment